В работе представлены 3 алгоритма по уменьшению масштаба сетей:

In [3]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm_notebook as tqdm
import networkx as nx 
import seaborn as sns

import os
path = 'C://pit'
filename_list = os.listdir(path)
graph_list = []
for filename in tqdm(filename_list):
    try:
        graph_list.append(nx.read_graphml(path+'//'+filename,
                              node_type=int))
    except Exception as e:
        print(e)
        print('In {}\n'.format(filename))

KeyboardInterrupt: 

In [ ]:
nodes_raw=[]       ###вершины 
dens=[]        ###плотность
degree=[]      ###степень вершин
edges=[]       ###cdzpb
               ### КОСТЫЛИ #####################
column_permutation = [2, 4, 0, 1, 3]
               #################################
    
for graph in graph_list:
    nodes_raw.append(nx.number_of_nodes(graph))
    dens.append(nx.density(graph))
    degree.append(nx.degree(graph))
    edges.append(nx.edges(graph))

nodes=np.array(nodes_raw).reshape(-1, 5)[:,column_permutation]
dens=np.array(dens).reshape(-1, 5)[:,column_permutation]
degree=np.array(degree).reshape(-1, 5)[:,column_permutation]
edges=np.array(edges).reshape(-1, 5)[:,column_permutation]

In [ ]:
# создаем на основе списка связей словарь для всех вершин, где значением будут вершины с которыми она связана    

from collections import defaultdict
compressed_edges_dict={}

def compress_edges_to_dict(edges):
    edges_dict=defaultdict(lambda : [])
    for element in edges:
        edges_dict[element[0]].append(element[1])
        edges_dict[element[1]].append(element[0])    
        edges_dict[element[0]] = list(set(edges_dict[element[0]]))
        edges_dict[element[1]] = list(set(edges_dict[element[1]]))
        
    # ищем в словоре ключи с одинаковыми значениями и объединяет найденные вершины
    return edges_dict, compressed_edges_dict
        
edges_dict, compressed_edges_dict = compress_edges_to_dict(graph_list[3].edges())

1. Алгоритм основан на замене вершин со степенью 2 свзязью.

In [ ]:
G = graph_list[3].copy()

nodes_with_two_degree_array = np.array(list(edges_dict))[np.array(list({key: len(value) for key, value in edges_dict.items()}.values())) == 2]

for node in nodes_with_two_degree_array:
    G.add_edges_from([(edges_dict[node])])
    G.remove_node(node)
    
G_degree = np.array([d for node, d in G.degree()]) 

print('количетво вершин = ', int(len(G_degree[G_degree > 0])))
print('распределение графа по степеням')
plt.figure(figsize=(20, 10))
plt.grid()    
plt.hist(np.array(G_degree[G_degree > 0]).flatten(), bins = np.linspace(0, 130, 300))
plt.show()


  2. Алгоритм объединяет k-clique community в одну вершину.

  3. Алгорим по объединению вершин с идентичным набором соседей
  